In [1]:
import torch

from config import get_config
from data_loader import *
from utils import prepare_dirs, save_config
from torch.autograd import Variable
from model import DenseNet
import torch.nn as nn

# for reloading modules
%load_ext autoreload
%autoreload 2

In [2]:
config, unparsed = get_config()

In [3]:
kwargs = {}
data_loader = get_train_valid_loader(config.data_dir,
            config.batch_size, config.augment, config.random_seed,
            config.valid_size, config.shuffle, config.show_sample,
            **kwargs)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_loader = data_loader[0]
valid_loader = data_loader[0]

In [5]:
num_blocks = config.num_blocks
num_layers_total = config.num_layers_total
growth_rate = config.growth_rate
bottleneck = config.bottleneck
theta = config.compression
dropout_rate = config.dropout_rate

In [6]:
epochs = config.epochs
lr = config.lr
momentum = config.momentum
weight_decay = config.weight_decay

In [7]:
model = DenseNet(num_blocks, 
                 num_layers_total,
                 growth_rate, 
                 10, 
                 bottleneck, 
                 dropout_rate, 
                 theta)

After Conv1: 16
	Layer 1 Input Channels: 16
	Layer 2 Input Channels: 28
	Layer 3 Input Channels: 40
	Layer 4 Input Channels: 52
	Layer 5 Input Channels: 64
	Layer 6 Input Channels: 76
	Layer 7 Input Channels: 88
	Layer 8 Input Channels: 100
	Layer 9 Input Channels: 112
	Layer 10 Input Channels: 124
	Layer 11 Input Channels: 136
	Layer 12 Input Channels: 148
After Dense Block 1: 160
After Transition Layer 1: 160
	Layer 1 Input Channels: 160
	Layer 2 Input Channels: 172
	Layer 3 Input Channels: 184
	Layer 4 Input Channels: 196
	Layer 5 Input Channels: 208
	Layer 6 Input Channels: 220
	Layer 7 Input Channels: 232
	Layer 8 Input Channels: 244
	Layer 9 Input Channels: 256
	Layer 10 Input Channels: 268
	Layer 11 Input Channels: 280
	Layer 12 Input Channels: 292
After Dense Block 2: 304
After Transition Layer 2: 304
	Layer 1 Input Channels: 304
	Layer 2 Input Channels: 316
	Layer 3 Input Channels: 328
	Layer 4 Input Channels: 340
	Layer 5 Input Channels: 352
	Layer 6 Input Channels: 364
	Laye

In [8]:
print('Number of model parameters: {:,}'.format(
            sum([p.data.nelement() for p in model.parameters()])))

Number of model parameters: 1,018,826


In [9]:
data_iter = iter(train_loader)
image, target = data_iter.next()

In [10]:
input_var = torch.autograd.Variable(image)
target_var = torch.autograd.Variable(target)

In [11]:
output = model(input_var)

Before Pool: torch.Size([64, 448, 8, 8])
After Pool: torch.Size([64, 448, 1, 1])
Before Flatten: torch.Size([64, 448])
After FC: torch.Size([64, 10])


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum, weight_decay=weight_decay)

In [13]:
loss = criterion(output, target_var)
# acc = accuracy(output, target)

In [16]:
predicted = torch.max(output.data, 1)[1]
total = len(target)
correct = (predicted == target).sum()
acc = 100 * (correct / total)
print("Accuracy: {}".format(acc))

Accuracy: 9.375


In [24]:
loss.data[0]

2.4496960639953613

In [21]:
def accuracy(predicted, ground_truth):
    """
    Params
    ------
    - predicted: torch.FloatTensor
    - ground_truth: torch.LongTensor
    """
    predicted = torch.max(predicted.data, 1)[1]
    total = len(ground_truth)
    correct = (predicted == ground_truth).sum()
    acc = 100 * (correct / total)
    return acc